<a href="https://colab.research.google.com/github/MonuSingh16/code-llms-genai/blob/main/transformers_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BART (Encoder-Decoder Style Method)

BART, or Bidirectional AutoRegressive Transformer found in this [paper](https://arxiv.org/pdf/1910.13461v1.pdf), is a Encoder-Decoder style model that leverages the traditional architecture found in the "Attention is All You Need" paper. They make a simple modification to the activation function from *ReLU to GeLU*.

This model excels at a number of tasks, including but not limited to: Machine Translation, Summarization, Categorization of Input Sentences, and Question Answering.

We'll showcase BART with a Text Summarization fine-tuning task today.

In [1]:
!pip install rouge-score evaluate transformers accelerate -qU

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 119.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.4 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, set_seed
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

import datasets
from datasets import load_metric, Dataset
from datasets import DatasetDict

 Let's load our model and tokenizer

In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

We'll be using the billsum dataset from Hugging Face which you can be found [here](https://huggingface.co/datasets/billsum/viewer/default).

Each of the rows contains a block of text from a legal bill - and then a plain english summary.

In [5]:
from datasets import load_dataset
dataset = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

We will create train/test/eval split of our dataset

In [7]:
import math

total_rows = 500
test_val_ratio = 0.2

val_rows = total_rows + math.floor(total_rows * test_val_ratio)
test_rows = val_rows + math.floor(total_rows * test_val_ratio)

subset_dataset = datasets.DatasetDict(
    {
        "train" : Dataset.from_dict(dataset[:total_rows]),
        "validation" : Dataset.from_dict(dataset[total_rows:val_rows]),
        "test" : Dataset.from_dict(dataset[val_rows:test_rows])
    }
)

In [8]:
subset_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 100
    })
})

We need to preprocess our data into tokenized representations.

These tokenized representations are what the model will actually see during training!

In [11]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
  model_inputs = tokenizer(
      examples["text"],
      max_length = max_input_length,
      truncation = True
  )
  labels = tokenizer(
      examples["summary"],
      max_length = max_target_length,
      truncation = True
  )
  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [12]:
tokenized_datasets = subset_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Removing all the unnecessary columns

In [14]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset.column_names)

We'll set up an evaluation pipeline that will help us monitor our model's performance!

In [15]:
!pip install nltk -qU

In [16]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
import evaluate

rouge_score = evaluate.load("rouge")

In [18]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: round(v, 4) for k, v in result.items()}

Now we can finally get to training!

We're going to train with the Seq2Seq objective as we're trying to convert one long sequence into a shorter sequence

In [19]:
batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_ckpt

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-CNN-DailyNews",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps)

In [20]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

In [21]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.608900,2.007145,0.174300,0.101800,0.157100,0.163500
2,2.059500,1.902250,0.182700,0.109000,0.164800,0.173100
3,1.799900,1.844106,0.188700,0.109300,0.167800,0.175700
4,1.633800,1.866578,0.187700,0.112100,0.167900,0.175200
5,1.484400,1.838645,0.185100,0.106100,0.163500,0.172600
6,1.390200,1.863310,0.189700,0.111300,0.168500,0.176500
7,1.291900,1.861291,0.183300,0.106000,0.163800,0.170800
8,1.259300,1.863719,0.182300,0.103800,0.163500,0.171400


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=504, training_loss=1.6826910518464588, metrics={'train_runtime': 771.3124, 'train_samples_per_second': 5.186, 'train_steps_per_second': 0.653, 'total_flos': 2438945832960000.0, 'train_loss': 1.6826910518464588, 'epoch': 8.0})